In [1]:
# !pip install pyaudio

In [2]:
import pyaudio
p = pyaudio.PyAudio()
for i in range(p.get_device_count()):
    print(i, p.get_device_info_by_index(i)['name'])
    # print(i, p.get_device_info_by_index(i))

0 HDA Intel PCH: ALC887-VD Analog (hw:0,0)
1 HDA Intel PCH: ALC887-VD Alt Analog (hw:0,2)
2 HDA Intel PCH: HDMI 0 (hw:0,3)
3 HDA Intel PCH: HDMI 1 (hw:0,7)
4 HDA Intel PCH: HDMI 2 (hw:0,8)
5 HDA NVidia: HDMI 0 (hw:1,3)
6 HDA NVidia: HDMI 1 (hw:1,7)
7 HDA NVidia: HDMI 2 (hw:1,8)
8 HDA NVidia: HDMI 3 (hw:1,9)
9 HDA NVidia: HDMI 4 (hw:1,10)
10 HDA NVidia: HDMI 5 (hw:1,11)
11 HDA NVidia: HDMI 6 (hw:1,12)
12 Plantronics Calisto 5200: USB Audio (hw:2,0)
13 sysdefault
14 front
15 surround40
16 surround51
17 surround71
18 hdmi
19 default
20 dmix


ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:877:(find_matching_chmap) Found no matching channel map


In [3]:
"Determine index of Micro - Calisto"

for i in range(p.get_device_count()):
    if "Plantronics" in p.get_device_info_by_index(i)['name']:
        index_device = i
        print(i, p.get_device_info_by_index(i))
        break

print(index_device)

12 {'index': 12, 'structVersion': 2, 'name': 'Plantronics Calisto 5200: USB Audio (hw:2,0)', 'hostApi': 0, 'maxInputChannels': 1, 'maxOutputChannels': 2, 'defaultLowInputLatency': 0.0239375, 'defaultLowOutputLatency': 0.007979166666666667, 'defaultHighInputLatency': 0.096, 'defaultHighOutputLatency': 0.032, 'defaultSampleRate': 48000.0}
12


In [4]:
import pyaudio
import wave

chunk = 1024 # Запись кусками по 1024 сэмпла
sample_format = pyaudio.paInt16 # 16 бит на выборку
channels = 1
rate = 16000 # Запись со скоростью 44100 выборок(samples) в секунду
seconds = 3
filename = "output_sound.wav"
p = pyaudio.PyAudio() # Создать интерфейс для PortAudio

stream = p.open(format=sample_format,
            channels=channels,
            rate=rate,
            frames_per_buffer=chunk,
            input_device_index=index_device, # индекс устройства с которого будет идти запись звука 
            input=True)

In [5]:
frames = [] 

In [6]:
for i in range(0, int(rate / chunk * seconds)):
    # print(i)
    data = stream.read(chunk, exception_on_overflow = False)
    frames.append(data)

In [7]:
stream.stop_stream()
stream.close()
p.terminate()

In [8]:
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(rate)
wf.writeframes(b''.join(frames))
wf.close()

In [33]:
filename = "output_sound.wav"

def listen_n_seconds(filename, input_device_index=11, chunk=1024, 
                            sample_format=pyaudio.paInt16, channels=1, 
                            rate=16000, seconds=3):

    p = pyaudio.PyAudio() # Создать интерфейс для PortAudio
    
    stream = p.open(format=sample_format,
            channels=channels,
            rate=rate,
            frames_per_buffer=chunk,
            input_device_index=input_device_index, # индекс устройства с которого будет идти запись звука 
            input=True)

    frames = [] 

    for i in range(0, int(rate / chunk * seconds)):
        data = stream.read(chunk, exception_on_overflow = False)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(rate)
    wf.writeframes(b''.join(frames))
    wf.close()

In [34]:
listen_n_seconds("output_sound1.wav", input_device_index=index_device)

In [35]:
import soundfile as sf

In [36]:
import urllib
import json

def asr_from_yandex_speeckit(audiofile_name, FOLDER_ID, API_KEY, language="ru-RU"):
    
    with open(audiofile_name, "rb") as f:
        data = f.read()

    params = "&".join([
        "topic=general",
        "folderId=%s" % FOLDER_ID,
        "lang=%s" % language
    ])
    
    url = urllib.request.Request("https://stt.api.cloud.yandex.net/speech/v1/stt:recognize?%s" % params, data=data)
    url.add_header("Authorization", "Api-Key %s" % API_KEY) # если API-ключ
    # url.add_header("Authorization", "Bearer %s" % IAM_TOKEN) # если IAM токен

    responseData = urllib.request.urlopen(url).read().decode('UTF-8')
    decodedData = json.loads(responseData)

    if decodedData.get("error_code") is None:
        asr_result = decodedData.get("result")
    else:
        asr_result = 'Nothing recognized'
    
    return asr_result

In [37]:
# from speeckit_tools import asr_from_yandex_speeckit

FOLDER_ID = "b1gami13b761380nb5hb" # Идентификатор каталога
API_KEY = "AQVN3XofRyHEW5PJOhBYCiObnGaZNYM8IvAQskdp" # API-ключ

wav_name = 'wav_file.ogg'
ogg_name = "ogg_file.ogg"

listen_n_seconds(wav_name, input_device_index=index_device)

data, samplerate = sf.read(wav_name)
sf.write(ogg_name, data, samplerate)

asr_result = asr_from_yandex_speeckit(ogg_name, FOLDER_ID, API_KEY)

asr_result

'1234-56-78'

In [24]:
counter = 0
ogg_name = "ogg_file.ogg"
wav_name = "wav_file.wav"

while counter < 6:
    ###listen micro
    listen_n_seconds(wav_name, input_device_index=index_device)
    data, samplerate = sf.read(wav_name)
    sf.write(ogg_name, data, samplerate)
    asr_result = asr_from_yandex_speeckit(ogg_name, FOLDER_ID, API_KEY)
    print(asr_result)
    
    counter +=1

Добросердечный 1
Товарный вагон 17
Что там еще было
Какую нибудь
Бостон динамикс

